# Monte Carlo Tree Search on Chess Game
In this notebook we demonstrate the Monte Carlo tree search applied to our chess implementation.
More specifically, starting from the initial game situation, we learn the best moves.

In [1]:
from chess import GameController, GameControllerAdapter, MCTS

In [2]:
def node_info(node):
    result = "" 
    result += f"{node.wins}/{node.simulations} (Wins/Simulations)\n"
    win_rate = node.wins/node.simulations if node.simulations != 0 else None
    result += f"{win_rate} (Wins/Simulations) \n"
    result += node.status()
    return result

In [3]:
ctl = GameController()
adapter = GameControllerAdapter(ctl)
mcts = MCTS(adapter)

In [3]:
mcts.run(n_simulations=1000, verbose=True, print_every=100)

0 simulations run.
100 simulations run.
200 simulations run.
300 simulations run.
400 simulations run.
500 simulations run.
600 simulations run.
700 simulations run.
800 simulations run.
900 simulations run.


# The best first move.

In [12]:
print(f"Root node wins/simulations: {mcts.node}")
print(f"Number of possible first moves: {len(mcts.node.children)}")
print("Best moves:")
for c in mcts.best_moves()[:4]:
    print(node_info(c))

Root node wins/simulations: Node level: 0
Wins: 505/1000
Number of possible first moves: 26
Ranking of moves:
43/67 (Wins/Simulations)
0.6417910447761194 (Wins/Simulations) 
 x . . . . . . o
 . . . . . . . .
 . . . . . . . .
 . . . . . . . .
 . . . . . . . .
 . . . . . . . .
 . . . . . . . .
 . . . . . . . x
Active Player: black
Winner: None
39/62 (Wins/Simulations)
0.6290322580645161 (Wins/Simulations) 
 o . . . . . . x
 . . . . . . . .
 . . . . . . . .
 . . . . . . . .
 . . . . . . . .
 . . . . . . . .
 . . . . . . . .
 x . . . . . . .
Active Player: black
Winner: None
30/52 (Wins/Simulations)
0.5769230769230769 (Wins/Simulations) 
 o . . . . . . o
 . . . . . . . .
 . . . . . . . .
 . . . . . . . .
 . . . . . . . .
 . . . . . . . .
 . . . . . . . .
 . . . x . . . x
Active Player: black
Winner: None
29/51 (Wins/Simulations)
0.5686274509803921 (Wins/Simulations) 
 o . . . . . . o
 x . . . . . . .
 . . . . . . . .
 . . . . . . . .
 . . . . . . . .
 . . . . . . . .
 . . . . . . . .
 . . 

In [16]:
mcts.node = mcts.node.rank_child_nodes()[0]

In [22]:
mcts.run(n_simulations=1000)

0 simulations run.
50 simulations run.
100 simulations run.
150 simulations run.
200 simulations run.
250 simulations run.
300 simulations run.
350 simulations run.
400 simulations run.
450 simulations run.
500 simulations run.
550 simulations run.
600 simulations run.
650 simulations run.
700 simulations run.
750 simulations run.
800 simulations run.
850 simulations run.
900 simulations run.
950 simulations run.


# The best move sequence learned from the initial game.
Thinking about the best initial move, MCTS also learns something about subsequent moves, alternating between best moves for the white and the black player. We see, however, that the depth and simulation number here is still limited.

In [14]:
# Play the best "game" learned so far (not necessarily up to a final game state).
node = mcts.node
while node.has_children():
    node = node.rank_child_nodes()[0]
    print(node_info(node))

43/67 (Wins/Simulations)
0.6417910447761194 (Wins/Simulations) 
 x . . . . . . o
 . . . . . . . .
 . . . . . . . .
 . . . . . . . .
 . . . . . . . .
 . . . . . . . .
 . . . . . . . .
 . . . . . . . x
Active Player: black
Winner: None
6/9 (Wins/Simulations)
0.6666666666666666 (Wins/Simulations) 
 x . . . . . . .
 . . . . . . . .
 . . . . . . . .
 . . . . . . . .
 . . . . . . . .
 . . . . . . . .
 . . . . . . . o
 . . . . . . . x
Active Player: white
Winner: None
1/1 (Wins/Simulations)
1.0 (Wins/Simulations) 
 x . . . . . . .
 . . . . . . . .
 . . . . . . . .
 . . . . . . . .
 . . . . . . . .
 . . . . . . . .
 . . . . . . . o
 . . . . . x . .
Active Player: black
Winner: None
0/1 (Wins/Simulations)
0.0 (Wins/Simulations) 
 x . . . . . . .
 . . . . . . . o
 . . . . . . . .
 . . . . . . . .
 . . . . . . . .
 . . . . . . . .
 . . . . . . . .
 . . . . . x . .
Active Player: white
Winner: None


# Play a full game.
Here we simulate a full game played through MCTS against itself. Starting from each chosen state, MCTS is retrained to select the subsequent best move, hence alternating between the white and the black player. Some observations we made here:
- Starting the game with only two towers per player, it never ends. This is expected unless the AI makes a mistake.
- Starting with the complete set of chess pieces per player, we see moves, that are much human-like (e.g., moving pieces somewhere were they cover a large area).
- However, some very obvious moves are not done by the AI. It seems that altogether the policy of selecting totally random moves in a playout (i.e., a simulation) is insufficient for determining the value of a game situation.
- For the same reason, the AI exhibits some risky, greedy behavior, e.g., moving to positions where it is closer to the opponent's king while ignoring that own pieces are under attack.
- Often, the value of beating an enemy piece that is not the king seems to be underestimated. The reason might be that there occurs too much randomness on the way down the (relatively deep) game tree. The behavior here might be too unrealistic and the number of possible moves to large.
- A larger number of simulations might be required (here we used 1000 per move).

In [4]:
# Reset current state to initial game in case this cell is rerun.
mcts.node = mcts.tree

# Run MCTS for current node. Find best move and rerun from there up to final game state.
while not mcts.node.state.is_final():
    mcts.run(n_simulations=1000, verbose=False)
    #mcts.node = mcts.node.best_child()
    mcts.node = sorted([c for c in mcts.node.children], reverse=True, key=lambda c: c.wins/c.simulations )[0]
    print(node_info(mcts.node))

55/86 (Wins/Simulations)
0.6395348837209303 (Wins/Simulations) 
 ♜ ♞ ♝ ♛ ♚ ♝ ♞ ♜
 ♟ ♟ ♟ ♟ ♟ ♟ ♟ ♟
 . . . . . . . .
 . . . . . . . .
 . . . . . . . .
 . . ♘ . . . . .
 ♙ ♙ ♙ ♙ ♙ ♙ ♙ ♙
 ♖ . ♗ ♕ ♔ ♗ ♘ ♖
Active Player: black
Winner: None
52/89 (Wins/Simulations)
0.5842696629213483 (Wins/Simulations) 
 ♜ ♞ ♝ ♛ ♚ ♝ ♞ ♜
 ♟ ♟ ♟ ♟ . ♟ ♟ ♟
 . . . . . . . .
 . . . . ♟ . . .
 . . . . . . . .
 . . ♘ . . . . .
 ♙ ♙ ♙ ♙ ♙ ♙ ♙ ♙
 ♖ . ♗ ♕ ♔ ♗ ♘ ♖
Active Player: white
Winner: None
49/81 (Wins/Simulations)
0.6049382716049383 (Wins/Simulations) 
 ♜ ♞ ♝ ♛ ♚ ♝ ♞ ♜
 ♟ ♟ ♟ ♟ . ♟ ♟ ♟
 . . . . . . . .
 . . . . ♟ . . .
 . . . . . . . .
 . . ♘ . ♙ . . .
 ♙ ♙ ♙ ♙ . ♙ ♙ ♙
 ♖ . ♗ ♕ ♔ ♗ ♘ ♖
Active Player: black
Winner: None
32/54 (Wins/Simulations)
0.5925925925925926 (Wins/Simulations) 
 ♜ ♞ ♝ ♛ ♚ ♝ . ♜
 ♟ ♟ ♟ ♟ ♞ ♟ ♟ ♟
 . . . . . . . .
 . . . . ♟ . . .
 . . . . . . . .
 . . ♘ . ♙ . . .
 ♙ ♙ ♙ ♙ . ♙ ♙ ♙
 ♖ . ♗ ♕ ♔ ♗ ♘ ♖
Active Player: white
Winner: None
25/43 (Wins/Simulations)
0.5813953488372093 (Wins/Simulations) 
